<a href="https://colab.research.google.com/github/H3N21QU3/Monografia/blob/main/Monografia_BI_Master_v02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
#Instalar biblioteca para visualizar dados para melhor entendimento. Plotar informação usando seaborn.
#Fonte: 
# https://medium.com/datadriveninvestor/building-neural-network-using-keras-for-classification-3a3656c726c1
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')
%config InlineBackend.figure_format = 'retina'


In [ ]:
#Importar arquivos diretamente da competição - entrar na competição primeiro!
#Inserir usuário e key recebida no arquivo .json entre aspas
import os, fnmatch
import zipfile
os.environ['KAGGLE_USERNAME'] = "calbuque" # copiar do kaggle.json
os.environ['KAGGLE_KEY'] = "1d095380238ab91098d1126f5344eaf3" # copiar do kaggle.json
!kaggle competitions download -c chh-ola -p chh-ola
#Os dados da competição ficarão no diretório /content/chh-ola
os.chdir('/content/chh-ola')

train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
SampleSubmission.csv: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#Filtrar e abrir arquivos .zip
listOfFiles = os.listdir('.')
pattern = "*.zip"
for file in listOfFiles:
    if fnmatch.fnmatch(file, pattern):
      print (file)
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
zip_ref.close()

train.csv.zip
test.csv.zip


In [ ]:
#Instalar biblioteca pandas_profiling se necessário
#!pip install pandas_profiling

In [ ]:
#Gerar numero aleatorio
import time
seconds = time.time()

In [ ]:
#Instalar bibliotecas
import pandas as pd
import numpy as np
import pandas_profiling as pp
#Importar arquivo de treino
dataset_train = pd.read_csv('train.csv')


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,4,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#Verifir tipos importados por coluna
dataset_train.dtypes

ID                          int64
vendor+AF8-id              object
pickup+AF8-loc            float64
drop+AF8-loc              float64
driver+AF8-tip             object
mta+AF8-tax                object
distance                  float64
pickup+AF8-time            object
drop+AF8-time              object
num+AF8-passengers        float64
toll+AF8-amount            object
payment+AF8-method        float64
rate+AF8-code             float64
stored+AF8-flag            object
extra+AF8-charges          object
improvement+AF8-charge     object
total+AF8-amount           object
dtype: object

In [ ]:
#Analisar informação importada
#vários casos de dado ausente e presença de sequencia de caracteres indesejada "+AF8-"
dataset_train.describe(include='all')

,ID,vendor+AF8-id,pickup+AF8-loc,drop+AF8-loc,driver+AF8-tip,mta+AF8-tax,distance,pickup+AF8-time,drop+AF8-time,num+AF8-passengers,toll+AF8-amount,payment+AF8-method,rate+AF8-code,stored+AF8-flag,extra+AF8-charges,improvement+AF8-charge,total+AF8-amount
count,1.048575e+06,1048575.0,1.048574e+06,1.048574e+06,1048573.0,1048574,1.048574e+06,1048574,1048574,1.048574e+06,1048573.0,1.048574e+06,1.048574e+06,1048574,1048574,1048573,1048573
unique,NaN,5.0,NaN,NaN,3578.0,4,NaN,323725,326900,NaN,490.0,NaN,NaN,2,15,3,6106
top,NaN,2.0,NaN,NaN,0.0,0.5,NaN,04/05/2017 11:14:23 PM,04/05/2017 12:00:00 AM,NaN,0.0,NaN,NaN,N,0,0.3,7.8
freq,NaN,549963.0,NaN,NaN,219673.0,1042799,NaN,16,38,NaN,929513.0,NaN,NaN,1041404,573702,1047768,24068
mean,5.242870e+05,NaN,1.648085e+02,1.626675e+02,NaN,NaN,2.859832e+00,NaN,NaN,1.590696e+00,NaN,1.317943e+00,1.043368e+00,NaN,NaN,NaN,NaN
std,3.026977e+05,NaN,6.579282e+01,6.953073e+01,NaN,NaN,3.709541e+00,NaN,NaN,1.253700e+00,NaN,4.853266e-01,5.621114e-01,NaN,NaN,NaN,NaN
min,0.000000e+00,NaN,1.000000e+00,1.000000e+00,NaN,NaN,0.000000e+00,NaN,NaN,0.000000e+00,NaN,1.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN
25%,2.621435e+05,NaN,1.250000e+02,1.130000e+02,NaN,NaN,9.300000e-01,NaN,NaN,1.000000e+00,NaN,1.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN
50%,5.242870e+05,NaN,1.620000e+02,1.620000e+02,NaN,NaN,1.600000e+00,NaN,NaN,1.000000e+00,NaN,1.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN
75%,7.864305e+05,NaN,2.330000e+02,2.330000e+02,NaN,NaN,2.900000e+00,NaN,NaN,2.000000e+00,NaN,2.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN


In [ ]:
#Importar dados de teste
dataset_test = pd.read_csv('test.csv')
dataset_test.dtypes   # nao tem total amount - variável a ser inferida
dataset_test['totalamount'] = 0.0 # colocando essa coluna para ter mesmas colunas do treinamento - facilita tratamento. 
                                  # Remover quando separar depois de tratamento

In [ ]:
# Limpeza de caracteres indesejados "+AF8-"
# !sed 's/\+AF8-//g' train.csv > train.csv
# Não utilizei -----------------------------------------------------------------------------------


In [ ]:
#Criar coluna que permita distinguir entre treino e teste. Ao concatenar as duas tabelas pode-se unificar o tratamento
dataset_train['treino'] = 1
dataset_test['treino'] = 0

In [ ]:
#Renomear colunas para facilitar analise
dataset_test.rename(columns={"vendor_id" : "vendorid"},inplace = True)
dataset_test.rename(columns={"pickup_loc" : "pickuploc"},inplace = True)
dataset_test.rename(columns={"drop_loc" : "droploc"},inplace = True)
dataset_test.rename(columns={"driver_tip" : "drivertip"},inplace = True)
dataset_test.rename(columns={"mta_tax" : "mtatax"},inplace = True)
dataset_test.rename(columns={"distance" : "distance"},inplace = True)
dataset_test.rename(columns={"pickup_time" : "pickuptime"},inplace = True)
dataset_test.rename(columns={"drop_time" : "droptime"},inplace = True)
dataset_test.rename(columns={"num_passengers" : "numpassengers"},inplace = True)
dataset_test.rename(columns={"toll_amount" : "tollamount"},inplace = True)
dataset_test.rename(columns={"payment_method" : "paymentmethod"},inplace = True)
dataset_test.rename(columns={"rate_code" : "ratecode"},inplace = True)
dataset_test.rename(columns={"stored_flag" : "storedflag"},inplace = True)
dataset_test.rename(columns={"extra_charges" : "extracharges"},inplace = True)
dataset_test.rename(columns={"improvement_charge" : "improvementcharge"},inplace = True)
#dataset_test.rename(columns={"totalamount" : "totalamount"},inplace = True)
dataset_test.dtypes

ID                     int64
vendorid               int64
pickuploc            float64
droploc              float64
drivertip            float64
mtatax               float64
distance             float64
pickuptime            object
droptime              object
numpassengers        float64
tollamount           float64
paymentmethod        float64
ratecode             float64
storedflag            object
extracharges         float64
improvementcharge    float64
totalamount          float64
treino                 int64
dtype: object

In [ ]:
#Verificar tamanho do dataset de teste de modo a detectar caso algum dado se perca
dataset_test.shape #teste não pode perder linhas

(154235, 18)

In [ ]:
#Renomear dataset de treino e igualar nomes com o dataset de teste

dataset_train.rename(columns={"vendor+AF8-id" : "vendorid"},inplace = True)
dataset_train.rename(columns={"pickup+AF8-loc" : "pickuploc"},inplace = True)
dataset_train.rename(columns={"drop+AF8-loc" : "droploc"},inplace = True)
dataset_train.rename(columns={"driver+AF8-tip" : "drivertip"},inplace = True)
dataset_train.rename(columns={"mta+AF8-tax" : "mtatax"},inplace = True)
dataset_train.rename(columns={"distance" : "distance"},inplace = True)
dataset_train.rename(columns={"pickup+AF8-time" : "pickuptime"},inplace = True)
dataset_train.rename(columns={"drop+AF8-time" : "droptime"},inplace = True)
dataset_train.rename(columns={"num+AF8-passengers" : "numpassengers"},inplace = True)
dataset_train.rename(columns={"toll+AF8-amount" : "tollamount"},inplace = True)
dataset_train.rename(columns={"payment+AF8-method" : "paymentmethod"},inplace = True)
dataset_train.rename(columns={"rate+AF8-code" : "ratecode"},inplace = True)
dataset_train.rename(columns={"stored+AF8-flag" : "storedflag"},inplace = True)
dataset_train.rename(columns={"extra+AF8-charges" : "extracharges"},inplace = True)
dataset_train.rename(columns={"improvement+AF8-charge" : "improvementcharge"},inplace = True)
dataset_train.rename(columns={"total+AF8-amount" : "totalamount"},inplace = True)
dataset_train.dtypes

ID                     int64
vendorid              object
pickuploc            float64
droploc              float64
drivertip             object
mtatax                object
distance             float64
pickuptime            object
droptime              object
numpassengers        float64
tollamount            object
paymentmethod        float64
ratecode             float64
storedflag            object
extracharges          object
improvementcharge     object
totalamount           object
treino                 int64
dtype: object

In [ ]:
#Concatenar teste e treino em dataframe unico
dataset_all = pd.concat([dataset_train, dataset_test], ignore_index=True)
dataset_all.dtypes

ID                     int64
vendorid              object
pickuploc            float64
droploc              float64
drivertip             object
mtatax                object
distance             float64
pickuptime            object
droptime              object
numpassengers        float64
tollamount            object
paymentmethod        float64
ratecode             float64
storedflag            object
extracharges          object
improvementcharge     object
totalamount           object
treino                 int64
dtype: object

In [ ]:
#Excluir datasets individuais de treino e teste
del(dataset_train)
del(dataset_test)

In [ ]:
#Verificar se dataframe possui valores anômalos
#Uma linha por vez teve comentário retirado e foi executada para analisar valores

dataset_all.vendorid.unique() #há valores diferentes de 1 e 2 - sujeira no meu dataframe ("+ACI-  +ACIAIg-status+ACIAIg- : 500+ACI-")
#dataset_all.drivertip.unique()
#dataset_all.mtatax.unique()
#dataset_all.tollamount.unique() #dados consertaveis
#dataset_all.extracharges.unique() #dados consertaveis
#dataset_all.improvementcharge.unique() #dados conserttáveis aqui tb
#dataset_all.totalamount.unique()


array([1, 2, '1', '2', '+ACI-  +ACIAIg-status+ACIAIg- : 500+ACI-'],
      dtype=object)

In [ ]:
# Analisar valores únicos em cada coluna para detectar anomalias (muitos linhas preenchidas com texto ao invés de número, ex: "+AC0-0.5 ")
# Fonte:
# https://towardsdatascience.com/10-python-pandas-tricks-that-make-your-work-more-efficient-2e8e483808ba
dataset_all.mtatax.value_counts()

0.5         1042799
0.5          153421
0              5269
0.0             759
+AC0-0.5        505
-0.5             55
0+AHs-            1
Name: mtatax, dtype: int64

In [ ]:
# Limpar dataset dos valores anômalos nas colunas numéricas - método: deleçao
# Não é viável reconstruir valor anterior
dataset_all ["vendorid"] = pd.to_numeric(dataset_all["vendorid"], errors="coerce")
dataset_all ["drivertip"] = pd.to_numeric(dataset_all["drivertip"], errors="coerce")
dataset_all ["mtatax"] = pd.to_numeric(dataset_all["mtatax"], errors="coerce")
dataset_all ["tollamount"] = pd.to_numeric(dataset_all["tollamount"], errors="coerce")
dataset_all ["extracharges"] = pd.to_numeric(dataset_all["extracharges"], errors="coerce")
dataset_all ["improvementcharge"] = pd.to_numeric(dataset_all["improvementcharge"], errors="coerce")
dataset_all ["totalamount"] = pd.to_numeric(dataset_all["totalamount"], errors="coerce")
dataset_all.dtypes

ID                     int64
vendorid             float64
pickuploc            float64
droploc              float64
drivertip            float64
mtatax               float64
distance             float64
pickuptime            object
droptime              object
numpassengers        float64
tollamount           float64
paymentmethod        float64
ratecode             float64
storedflag            object
extracharges         float64
improvementcharge    float64
totalamount          float64
treino                 int64
dtype: object

In [ ]:
#Converter colunas 'pickuptime' e 'droptime' de objeto para formato datetime 
# Fonte:
# https://www.geeksforgeeks.org/convert-the-column-type-from-string-to-datetime-format-in-pandas-dataframe/
dataset_all['pickuptime']= pd.to_datetime(dataset_all['pickuptime']) 
dataset_all['droptime']= pd.to_datetime(dataset_all['droptime'])

In [ ]:
#Agrupar valores únicos para examinar eventuais valores inadequados
#Cada linha teve comentário removido uma de cada vez e resultado analisado

#dataset_all.vendorid.unique()
#dataset_all.drivertip.unique()
#dataset_all.mtatax.unique()       #array([1., 0.]) - converter para categoria e depois one hot encoder
#dataset_all.tollamount.unique()
#dataset_all.extracharges.unique()
dataset_all.improvementcharge.unique() #array([1., 0.]) - converter para categoria e depois one hot encoder
#dataset_all.totalamount.unique()
#dataset_all.pickuptime.unique()
#dataset_all.droptime.unique()
#dataset_all.paymentmethod.unique()  #array([1., 2., 3., 4.]) - converter para categoria e depois one hot encoder
#dataset_all.ratecode.unique()  #array([ 1.,  2.,  3.,  4.,  5., 99.,  6.]) - converter para categoria e depois one hot encoder
#dataset_all.pickupmonth.unique()
#dataset_all.head

array([ 0.3,  nan,  0. , -0.3])

In [ ]:
#Extrair dia, mês e ano das colunas com tipo "datetime", criando novas colunas para inserir resultado
#Extrair hora e dia da semana em busca de um possível padrão

dataset_all['pickupday'] = dataset_all['pickuptime'].dt.day #extrair dia e colocar na coluna nova
dataset_all['pickupmonth'] = dataset_all['pickuptime'].dt.month #extrair mes e colocar na coluna nova
dataset_all['pickupyear'] = dataset_all['pickuptime'].dt.year #extrair year e colocar na coluna nova
dataset_all['pickuphourinsecond'] = (dataset_all['pickuptime'].dt.hour*60 + dataset_all['pickuptime'].dt.minute)*60 + dataset_all['pickuptime'].dt.second
dataset_all['pickupdayofweek'] = dataset_all['pickuptime'].dt.dayofweek

#dataset_all.pickupyear.head
dataset_all['dropday'] = dataset_all['droptime'].dt.day #extrair dia e colocar na coluna nova
dataset_all['dropmonth'] = dataset_all['droptime'].dt.month #extrair mes e colocar na coluna nova
dataset_all['dropyear'] = dataset_all['droptime'].dt.year #extrair year e colocar na coluna nova
dataset_all['drophourinsecond'] = (dataset_all['droptime'].dt.hour*60 + dataset_all['droptime'].dt.minute)*60 + dataset_all['droptime'].dt.second

#Realização de cálculo a partir das informações disponíveis - duração da viagem
dataset_all['timedeltas'] = dataset_all['droptime'] - dataset_all['pickuptime']  #duração da viagem
dataset_all['timedeltas'] = dataset_all['timedeltas'].dt.total_seconds() #converter para float !!

dataset_all.timedeltas.head


<bound method NDFrame.head of 0           321.0
1           990.0
2           377.0
3           968.0
4           162.0
            ...  
1202805     960.0
1202806     420.0
1202807     285.0
1202808     802.0
1202809    1014.0
Name: timedeltas, Length: 1202810, dtype: float64>

In [ ]:
# Excluir colunas com tipo "datetime"

dataset_all = dataset_all.drop("pickuptime", axis=1) #axis=1 --> column
dataset_all = dataset_all.drop("droptime", axis=1) #axis=1 --> column
dataset_all.dtypes



ID                      int64
vendorid              float64
pickuploc             float64
droploc               float64
drivertip             float64
mtatax                float64
distance              float64
numpassengers         float64
tollamount            float64
paymentmethod         float64
ratecode              float64
storedflag             object
extracharges          float64
improvementcharge     float64
totalamount           float64
treino                  int64
pickupday             float64
pickupmonth           float64
pickupyear            float64
pickuphourinsecond    float64
pickupdayofweek       float64
dropday               float64
dropmonth             float64
dropyear              float64
drophourinsecond      float64
timedeltas            float64
dtype: object

In [ ]:
# Excluir linhas com algum dado vazio
dataset_all.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

# Excluir linhas com valor 'nan'. 
dataset_all.dropna(inplace = True) 


In [ ]:
# Transformar colunas com valor não numérico (categórico) em one hot encoder
# Uma das colunas é excluída por ter alta correlação com as restantes drop_first=True
# Fonte:
# http://www.insightsbot.com/blog/zuyVu/python-one-hot-encoding-with-pandas-made-simple
dataset_all['vendorid'] = pd.Categorical(dataset_all['vendorid']) # define coluna como tipo categoria
dfTemp = pd.get_dummies(dataset_all['vendorid'], prefix = 'cat', drop_first=True) # faz a transformação da coluna em varias e salva em outro DF
dataset_all = pd.concat([dataset_all, dfTemp], axis=1) # junta os dois dataframes
dataset_all = dataset_all.drop("vendorid", axis=1) #axis=1 --> column

In [ ]:
dataset_all['storedflag'] = pd.Categorical(dataset_all['storedflag']) # define coluna como tipo categoria
dfTemp = pd.get_dummies(dataset_all['storedflag'], prefix = 'flag', drop_first=True) # faz a transformação da coluna em varias e salva em outro DF
dataset_all = pd.concat([dataset_all, dfTemp], axis=1) # junta os dois dataframes
dataset_all = dataset_all.drop("storedflag", axis=1) #axis=1 --> column

In [ ]:
dataset_all['mtatax'] = pd.Categorical(dataset_all['mtatax']) # define coluna como tipo categoria
dfTemp = pd.get_dummies(dataset_all['mtatax'], prefix = 'mtatax', drop_first=True) # faz a transformação da coluna em varias e salva em outro DF
dataset_all = pd.concat([dataset_all, dfTemp], axis=1) # junta os dois dataframes
dataset_all = dataset_all.drop("mtatax", axis=1) #axis=1 --> column #apaga coluna antiga

In [ ]:
dataset_all['improvementcharge'] = pd.Categorical(dataset_all['improvementcharge']) # define coluna como tipo categoria
dfTemp = pd.get_dummies(dataset_all['improvementcharge'], prefix = 'improvmtcharge', drop_first=True) # faz a transformação da coluna em varias e salva em outro DF
dataset_all = pd.concat([dataset_all, dfTemp], axis=1) # junta os dois dataframes
dataset_all = dataset_all.drop("improvementcharge", axis=1) #axis=1 --> column #apaga coluna antiga

In [ ]:
dataset_all['paymentmethod'] = pd.Categorical(dataset_all['paymentmethod']) # define coluna como tipo categoria
dfTemp = pd.get_dummies(dataset_all['paymentmethod'], prefix = 'paymethod', drop_first=True) # faz a transformação da coluna em varias e salva em outro DF
dataset_all = pd.concat([dataset_all, dfTemp], axis=1) # junta os dois dataframes
dataset_all = dataset_all.drop("paymentmethod", axis=1) #axis=1 --> column #apaga coluna antiga

In [ ]:
dataset_all['ratecode'] = pd.Categorical(dataset_all['ratecode']) # define coluna como tipo categoria
dfTemp = pd.get_dummies(dataset_all['ratecode'], prefix = 'ratecode', drop_first=True) # faz a transformação da coluna em varias e salva em outro DF
dataset_all = pd.concat([dataset_all, dfTemp], axis=1) # junta os dois dataframes
dataset_all = dataset_all.drop("ratecode", axis=1) #axis=1 --> column #apaga coluna antiga

In [ ]:
dataset_all['pickupdayofweek'] = pd.Categorical(dataset_all['pickupdayofweek']) # define coluna como tipo categoria
dfTemp = pd.get_dummies(dataset_all['pickupdayofweek'], prefix = 'pickupdayofweek', drop_first=True) # faz a transformação da coluna em varias e salva em outro DF
dataset_all = pd.concat([dataset_all, dfTemp], axis=1) # junta os dois dataframes
dataset_all = dataset_all.drop("pickupdayofweek", axis=1) #axis=1 --> column #apaga coluna antiga

In [ ]:
#Excluindo colunas que não serão mais utilizadas

dataset_all = dataset_all.drop("pickupmonth", axis=1) #axis=1 --> base abrange mesmo mês e ano
dataset_all = dataset_all.drop("pickupyear", axis=1) #axis=1 --> base abrange mesmo mês e ano
dataset_all = dataset_all.drop("dropmonth", axis=1) #axis=1 --> base abrange mesmo mês e ano
dataset_all = dataset_all.drop("dropyear", axis=1) #axis=1 --> base abrange mesmo mês e ano
dataset_all = dataset_all.drop("pickuploc", axis=1) #axis=1 --> número não representa informação útil para modelo - sendo suplantando pela info de distância
dataset_all = dataset_all.drop("droploc", axis=1) #axis=1 --> número não representa informação útil para modelo - sendo suplantando pela info de distância


In [ ]:
#Visualizar dataframe resultante
dataset_all

,ID,drivertip,distance,numpassengers,tollamount,extracharges,totalamount,treino,pickupday,pickuphourinsecond,dropday,drophourinsecond,timedeltas,cat_2.0,flag_Y,mtatax_0.0,mtatax_0.5,improvmtcharge_0.0,improvmtcharge_0.3,paymethod_2.0,paymethod_3.0,paymethod_4.0,ratecode_2.0,ratecode_3.0,ratecode_4.0,ratecode_5.0,ratecode_6.0,ratecode_99.0,pickupdayofweek_1.0,pickupdayofweek_2.0,pickupdayofweek_3.0,pickupdayofweek_4.0,pickupdayofweek_6.0
0,0,1.83,0.70,1.0,0.0,1.0,9.13,1,4.0,64783.0,4.0,65104.0,321.0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,1,3.56,4.64,1.0,0.0,1.0,21.36,1,3.0,68614.0,3.0,69604.0,990.0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,1.50,1.29,2.0,0.0,0.0,8.80,1,3.0,54373.0,3.0,54750.0,377.0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,1.50,2.74,1.0,0.0,0.0,14.80,1,4.0,29452.0,4.0,30420.0,968.0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,4,0.00,0.45,6.0,0.0,0.0,4.80,1,5.0,50579.0,5.0,50741.0,162.0,1,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202805,154230,2.56,1.47,6.0,0.0,1.0,0.00,0,5.0,69449.0,5.0,70409.0,960.0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1202806,154231,1.00,1.10,2.0,0.0,0.0,0.00,0,5.0,46930.0,5.0,47350.0,420.0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1202807,154232,1.56,1.26,1.0,0.0,1.0,0.00,0,6.0,71608.0,6.0,71893.0,285.0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1202808,154233,2.45,2.20,1.0,0.0,0.0,0.00,0,3.0,39178.0,3.0,39980.0,802.0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:

#sns.pairplot(dataset_all, hue='totalamount')  #não funcionou

In [ ]:
#Gerar heatmap
#Fonte:
# https://likegeeks.com/seaborn-heatmap-tutorial/
	
heat_map = sns.heatmap(dataset_all.corr(), annot=False)
plt.show();

In [ ]:
# Remover colunas com alta correlacao
# Compara as colunas automaticamente uma com as outras - remove as de alta correlação
removeAlta = True # Fonte: https://towardsdatascience.com/feature-selection-correlation-and-p-value-da8921bfb3cf
if removeAlta:
  corr = dataset_all.corr()
  columns = np.full((corr.shape[0],), True, dtype=bool)
  for i in range(corr.shape[0]):
     for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.9:
            print("Correlacao maior q 0.9 entre " + dataset_all.columns[i] + " e " + dataset_all.columns[j] + " vou remover " + dataset_all.columns[j])
            if columns[j]:
               columns[j] = False 
               if dataset_all.columns[j] == "totalamount": columns[j] = True  #não vou apagar minha variavel dependente
                
  #Todas as colunas marcadas com falso não serão copiadas na versão final do dataset
  selected_columns = dataset_all.columns[columns] #fitra as colunas com "verdadeiro"
  dataset_all = dataset_all[selected_columns]


Correlacao maior q 0.9 entre pickupday e dropday vou remover dropday
Correlacao maior q 0.9 entre pickuphourinsecond e drophourinsecond vou remover drophourinsecond


In [ ]:
#Investigar correlação/relevância das novas colunas geradas e valor a ser inferido
#Retirar o comentário de uma linha por vez

dataset_all['totalamount'].corr(dataset_all['timedeltas'])
#dataset_all['totalamount'].corr(dataset_all['cat_2.0'])
#dataset_all['totalamount'].corr(dataset_all['flag_Y'])
#dataset_all.shape[1]
#dataset_all.columns[3]
#dataset_all['totalamount'].corr(dataset_all[dataset_all.columns[3]])

0.15543154633372144

In [ ]:
#Eliminar colunas que possuem pouca correlação com totalamount
eliminar = True

if eliminar:
  droplist = []
  for i in range(dataset_all.shape[1]):
    if dataset_all['totalamount'].corr(dataset_all[dataset_all.columns[i]]) < 0.01:
      print (dataset_all.columns[i])
      droplist.append(dataset_all.columns[i])
      
  dataset_all.drop(columns=droplist, inplace=True)

pickupday
pickuphourinsecond
mtatax_0.5
improvmtcharge_0.0
improvmtcharge_0.3
paymethod_2.0
paymethod_3.0
paymethod_4.0
ratecode_6.0
ratecode_99.0
pickupdayofweek_1.0
pickupdayofweek_2.0
pickupdayofweek_3.0
pickupdayofweek_4.0
pickupdayofweek_6.0


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
# Separar em dois grupos: treino e teste conforme distribuição original.

set_train = dataset_all[dataset_all['treino'] == 1].copy()
set_test = dataset_all[dataset_all['treino'] == 0].copy()

set_train.drop(columns=['treino'], inplace=True)
set_test.drop(columns=['treino'], inplace=True)

In [ ]:
set_test  #havia 154235 linhas

,ID,drivertip,distance,numpassengers,tollamount,extracharges,totalamount,timedeltas,cat_2.0,flag_Y,mtatax_0.0,ratecode_2.0,ratecode_3.0,ratecode_4.0,ratecode_5.0
1048575,0,1.00,0.70,1.0,0.0,0.5,0.0,257.0,0,0,0,0,0,0,0
1048576,1,2.55,1.00,1.0,0.0,0.0,0.0,1113.0,0,0,0,0,0,0,0
1048577,2,2.96,3.35,2.0,0.0,0.0,0.0,892.0,1,0,0,0,0,0,0
1048578,3,2.00,0.80,2.0,0.0,0.0,0.0,347.0,0,0,0,0,0,0,0
1048579,4,1.00,0.70,1.0,0.0,0.0,0.0,413.0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202805,154230,2.56,1.47,6.0,0.0,1.0,0.0,960.0,1,0,0,0,0,0,0
1202806,154231,1.00,1.10,2.0,0.0,0.0,0.0,420.0,0,0,0,0,0,0,0
1202807,154232,1.56,1.26,1.0,0.0,1.0,0.0,285.0,1,0,0,0,0,0,0
1202808,154233,2.45,2.20,1.0,0.0,0.0,0.0,802.0,0,0,0,0,0,0,0


In [ ]:
# Analise de correlacao
import seaborn as sb
pearsoncorr = set_train.corr(method='pearson')
sb.heatmap(pearsoncorr, 
            xticklabels=pearsoncorr.columns,
            yticklabels=pearsoncorr.columns,
            cmap='RdBu_r',
            annot=True,
            linewidth=0.5)

In [ ]:
# Separar coluna a ser inferida do dataframe de treino. Criar a variável dependente.
# Y nao pode estar escalonado .... deixar valores originais.
# eliminei colunas e totalamount mudou de lugar
Y = set_train.iloc[:, (set_train.columns.get_loc("totalamount"))] #tirar .values no final para não perder dataframe

Y = pd.DataFrame(Y, columns = ['totalamount'])  #transformando num dataframe a variavel dependente -  

Y

,totalamount
0,9.13
1,21.36
2,8.80
3,14.80
4,4.80
...,...
1048570,9.80
1048571,18.30
1048572,14.15
1048573,11.80


In [ ]:
#Retirar a variável dependente do dataframe de treino.

X = set_train.drop("totalamount", axis=1) #axis=1 --> column - Novo dataframe = antigo sem coluna de respostas
X

,ID,drivertip,distance,numpassengers,tollamount,extracharges,timedeltas,cat_2.0,flag_Y,mtatax_0.0,ratecode_2.0,ratecode_3.0,ratecode_4.0,ratecode_5.0
0,0,1.83,0.70,1.0,0.0,1.0,321.0,0,0,0,0,0,0,0
1,1,3.56,4.64,1.0,0.0,1.0,990.0,1,0,0,0,0,0,0
2,2,1.50,1.29,2.0,0.0,0.0,377.0,1,0,0,0,0,0,0
3,3,1.50,2.74,1.0,0.0,0.0,968.0,1,0,0,0,0,0,0
4,4,0.00,0.45,6.0,0.0,0.0,162.0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,1048570,1.00,1.40,1.0,0.0,0.5,475.0,0,0,0,0,0,0,0
1048571,1048571,0.00,3.60,1.0,0.0,0.0,1397.0,0,0,0,0,0,0,0
1048572,1048572,2.35,1.70,1.0,0.0,0.5,802.0,0,0,0,0,0,0,0
1048573,1048573,0.00,1.80,4.0,0.0,1.0,767.0,1,0,0,0,0,0,0


In [ ]:
# Normalização dos valores entre zero e um

from sklearn.preprocessing import MinMaxScaler
## abaixo vai colocar valores entre zero e um
sc = MinMaxScaler(feature_range = (0, 1))


In [ ]:
#Gerar número aleatorio para sorteio de treino e teste
import time
seconds = int(time.time() - seconds)
seconds


302

In [ ]:
# Dividindo o dataset em treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = seconds)  

In [ ]:

    #Step 1: fit the scaler on the TRAINING data
    #Step 2: use the scaler to transform the training data
    #Step 3: use the transformed training data to fit the predictive model
    #Step 4: use the scaler to transform the TEST data
    #Step 5: predict using the trained model and the transformed TEST data

    #Fonte:
    #https://stackoverflow.com/questions/50565937/how-to-normalize-the-train-and-test-data-using-minmaxscaler-sklearn

In [ ]:
#Normalizar dados de treino

X_train = sc.fit_transform(X_train)


In [ ]:
#Normalizar dados de teste

X_test = sc.transform(X_test)

In [ ]:
y_test

,totalamount
10373,19.55
809330,7.80
15349,70.27
765960,22.25
606849,15.95
...,...
326278,6.96
198804,9.96
656601,20.80
1032912,11.80


In [ ]:
# Seleção de regressor (escolher apenas um como True)
Random_Forest = False
Xgboost = False
GradientBoost = True
AdaBoost = False
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
# Importando ferramentas e modelos necessários para Random Forest
if Random_Forest :
  from sklearn.ensemble import RandomForestRegressor
  

  params = {
      'n_estimators' : 5,
      'n_jobs' : -1,
      'verbose' : 1,
  }



In [ ]:
# Ajustar automaticamente o modelo para os melhores parametros
if Random_Forest :
  mae = 1 #valor inicial de mean absolute error
  for x in range(1,10):
    paramstmp = {
      'n_estimators' : x,
      'n_jobs' : -1,
      'verbose' : 1,
    }    
    ## Chamada para regressor linear
    lin_regtmp = RandomForestRegressor(**paramstmp)
    ## ajustar treino e dados de teste. 
    lin_regtmp.fit(X_train, y_train) #treinar o x de treino escalonado em relação ao y de treino
    ## Predizer dados de treino  
    y_predtmp = lin_regtmp.predict(X_test)  #predizer usando o x de teste escalonado
    maetemp = mean_absolute_error(y_predtmp, y_test)
    if maetemp < mae: #comparar o predito com o dado real
      mae = maetemp # valor mais otimizado foi achado
      lin_reg = lin_regtmp
      params = paramstmp
      y_pred = y_predtmp
      print('\n')
      print('novo minimo!')
    print("Mean Absolute Error of x=" + str(x) + " : " + str(maetemp))
    print('\n')
  

In [ ]:
# Excecutando XgBoost
# https://www.kaggle.com/dansbecker/xgboost

if Xgboost :
  from xgboost import XGBRegressor
  lin_reg = XGBRegressor()
  # Add silent=True to avoid printing out updates with each cycle
  lin_reg.fit(X_train, y_train, verbose=True)
  # make predictions
  y_pred = lin_reg.predict(X_test)

  from sklearn.metrics import mean_absolute_error
  print("Mean Absolute Error : " + str(mean_absolute_error(y_pred, y_test)))


In [ ]:
 # Executando Gradient Boost
if GradientBoost :  
  from sklearn.datasets import make_regression
  from sklearn.ensemble import GradientBoostingRegressor
  params = {'n_estimators': 500,
          'max_depth': 6,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'ls'}
  mae = 2 #valor inicial de mean absolute error
  for x in range(700,800, 100):
    paramstmp = {'n_estimators': x,
          'max_depth': 12,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'ls'}
    ## Chamada para regressor linear
    lin_regtmp = GradientBoostingRegressor(**paramstmp)
    ## ajustar treino e dados de teste. 
    lin_regtmp.fit(X_train, y_train) #treinar o x de treino escalonado em relação ao y de treino
    ## Predizer dadis de treino  
    y_predtmp = lin_regtmp.predict(X_test)  #predizer usando o x de teste escalonado
    maetemp = mean_absolute_error(y_predtmp, y_test)
    if maetemp < mae: #comparar o predito com o dado real
      mae = maetemp # valor mais otimizado foi achado
      lin_reg = lin_regtmp
      params = paramstmp
      y_pred = y_predtmp
      print('\n')
      print('novo minimo!')
    print("Mean Absolute Error of x=" + str(x) + " : " + str(maetemp))
    print('\n')

# Testes anteriores (maxdepth = 4):
#Mean Absolute Error of x=400 : 0.8411798023716722
#Mean Absolute Error of x=500 : 0.7363074561344162
#Mean Absolute Error of x=600 : 0.6788078831212778
#Mean Absolute Error of x=700 : 0.649170356014943
#Mean Absolute Error of x=1200 : 0.5665202175385219
#(maxdepth = 4)
#Mean Absolute Error of x=500 : 0.529386211795098
#(maxdepth = 6)
#Mean Absolute Error of x=500 : 0.3604175543475147
#(maxdepth = 12)
#Mean Absolute Error of x=700 : 0.3394432780996914  

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)




novo minimo!
Mean Absolute Error of x=700 : 0.3394432780996914




In [ ]:
 # Executando AdaBoost
if AdaBoost :
  from sklearn.ensemble import AdaBoostRegressor
  mae = 10 #valor inicial de mean absolute error
  for x in range(97,106,3):    
    ## Chamada para regressor linear
    lin_regtmp = AdaBoostRegressor(random_state=0, n_estimators=x, learning_rate = 0.1)
    ## ajustar treino e dados de teste. 
    lin_regtmp.fit(X_train, y_train) #treinar o x de treino escalonado em relação ao y de treino
    ## Predizer dadis de treino  
    y_predtmp = lin_regtmp.predict(X_test)  #predizer usando o x de teste escalonado
    maetemp = mean_absolute_error(y_predtmp, y_test)
    if maetemp < mae: #comparar o predito com o dado real
      mae = maetemp # valor mais otimizado foi achado
      lin_reg = lin_regtmp
      params = paramstmp
      y_pred = y_predtmp
      print('\n')
      print('novo minimo!')
    print("Mean Absolute Error of x=" + str(x) + " : " + str(maetemp))
    print('\n')

In [ ]:
y_pred

array([19.60716284,  7.95461658, 70.2429938 , ..., 20.16879236,
       11.59657787, 12.44110901])

In [ ]:
X


,ID,drivertip,distance,numpassengers,tollamount,extracharges,timedeltas,cat_2.0,flag_Y,mtatax_0.0,ratecode_2.0,ratecode_3.0,ratecode_4.0,ratecode_5.0
0,0,1.83,0.70,1.0,0.0,1.0,321.0,0,0,0,0,0,0,0
1,1,3.56,4.64,1.0,0.0,1.0,990.0,1,0,0,0,0,0,0
2,2,1.50,1.29,2.0,0.0,0.0,377.0,1,0,0,0,0,0,0
3,3,1.50,2.74,1.0,0.0,0.0,968.0,1,0,0,0,0,0,0
4,4,0.00,0.45,6.0,0.0,0.0,162.0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,1048570,1.00,1.40,1.0,0.0,0.5,475.0,0,0,0,0,0,0,0
1048571,1048571,0.00,3.60,1.0,0.0,0.0,1397.0,0,0,0,0,0,0,0
1048572,1048572,2.35,1.70,1.0,0.0,0.5,802.0,0,0,0,0,0,0,0
1048573,1048573,0.00,1.80,4.0,0.0,1.0,767.0,1,0,0,0,0,0,0


In [ ]:
#Aplicar modelo no grupo de teste e salvar previsao como saida

######## ACHO - usar mesmo scaler para tudo
#sc2 = MinMaxScaler()
#XNorm = sc2.fit_transform(X) #normalizar todo meu X - parametros para modelo completo

# do arquivo test.csv - para fazer minha previsao.... como juntei pra tratar, coloquei uma coluna totalamount falsa
# se a coluna ainda estiver lá, eu excluo ela
if 'totalamount' in set_test.columns:
  set_test = set_test.drop("totalamount", axis=1) # Novo dataframe = antigo sem coluna totalamount

#vou fazer um modelo de tudo... sem separar em dois grupos
######## ACHO remover essa linha.......... ou vai treinar de novo.... tem que usar o que já treinei
#lin_reg = RandomForestRegressor(**params)
## fit train and test data. 
#lin_reg.fit(XNorm, Y) #.flatten() precisa, mas nao aceita

set_testNorm = sc.transform(set_test) #normalizar meu X que devo prever para enviar -- do arquivo test.csv
predicAnswer = lin_reg.predict(set_testNorm)
predicAnswer

array([ 7.28791206, 15.26248621, 17.54747008, ...,  9.32346649,
       14.49501933, 15.99666603])

In [ ]:
#Colocar resultado no formato de arquivo para envio ao Kaggle
result = []
for idx in range(set_test.shape[0]):
  result.append([idx, predicAnswer[idx]])
result = pd.DataFrame(result, columns=['ID', 'total_amount'])
result.to_csv('result.csv', index=False)

In [ ]:
result

,ID,total_amount
0,0,7.287912
1,1,15.262486
2,2,17.547470
3,3,8.431631
4,4,7.961609
...,...,...
154230,154230,15.342495
154231,154231,8.499292
154232,154232,9.323466
154233,154233,14.495019


In [ ]:
!ls

chh-ola     SampleSubmission.csv  test.csv.zip	train.csv.zip
result.csv  test.csv		  train.csv
